In [20]:
import geopandas as gpd
from shapely.geometry import Point
from shapely.ops import nearest_points
import pandas as pd

## Load the dataset

In [21]:
train_gdf  = gpd.read_file(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\PhD\Mapping\ABC\ABC-model-18-02-2024\results\n_01\inputs\n_1_train.shp")
metro_gdf = gpd.read_file(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\PhD\Mapping\ABC\ABC-model-18-02-2024\results\n_01\inputs\n_1_metro.shp")
tram_gdf = gpd.read_file(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\PhD\Mapping\ABC\ABC-model-18-02-2024\results\n_01\inputs\n_1_tram.shp")
bus_gdf = gpd.read_file(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\PhD\Mapping\ABC\ABC-model-18-02-2024\results\n_01\inputs\n_1_bus.shp")
bus_tranne_90_gdf= gpd.read_file(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\PhD\Mapping\ABC\ABC-model-18-02-2024\results\n_01\inputs\n_1_bus_not_90.shp")
regional_bus_gdf= gpd.read_file(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\PhD\Mapping\ABC\ABC-model-18-02-2024\results\n_01\inputs\n_1_reg_bus.shp")

## load the GTFS data 

In [3]:
#ATM
calendar_df = pd.read_csv("D:\\15_Polimi\\Polimi_onedrive\\OneDrive - Politecnico di Milano\\Lombardia GTFS\\ATM\\calendar.txt")
trips_df= pd.read_csv("D:\\15_Polimi\\Polimi_onedrive\\OneDrive - Politecnico di Milano\\Lombardia GTFS\\ATM\\trips.txt")
stop_time_df= pd.read_csv(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\Lombardia GTFS\ATM\stop_times.txt")
# Filter for services that are available on Wednesday
wednesday_services = calendar_df[calendar_df['wednesday'] == 1]
wednesday_trips = trips_df[trips_df['service_id'].isin(wednesday_services['service_id'])]
wedndesday_stops_atm= stop_time_df[stop_time_df['trip_id'].isin(wednesday_trips['trip_id'])]

In [4]:
# train
calendar_df = pd.read_csv(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\Lombardia GTFS\TrenitaliaTrenord\calendar.txt")
# Filter for services that are available on Wednesday
wednesday_services = calendar_df[calendar_df['wednesday'] == 1]
# Extract the list of service_id values for Wednesday
wednesday_service_ids = wednesday_services['service_id'].tolist()
# Load the trips.txt file
trips_df = pd.read_csv(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\Lombardia GTFS\TrenitaliaTrenord\trips.txt")
# Filter trips by those that have a service_id that operates on Wednesday
wednesday_trips = trips_df[trips_df['service_id'].isin(wednesday_service_ids)]
# Load the stop_times.txt file
stop_times_df = pd.read_csv(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\Lombardia GTFS\TrenitaliaTrenord\stop_times.txt")
# Filter stop_times to only include rows with trip_id that are in wednesday_trips
wednesday_stop_times_train = stop_times_df[stop_times_df['trip_id'].isin(wednesday_trips['trip_id'])]

In [5]:
# others
calendar_df = pd.read_csv(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\Lombardia GTFS\AltriGestori\calendar.txt")
# Filter for services that are available on Wednesday
wednesday_services = calendar_df[calendar_df['wednesday'] == 1]
# Extract the list of service_id values for Wednesday
wednesday_service_ids = wednesday_services['service_id'].tolist()
# Load the trips.txt file
trips_df = pd.read_csv(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\Lombardia GTFS\AltriGestori\trips.txt")
# Filter trips by those that have a service_id that operates on Wednesday
wednesday_trips = trips_df[trips_df['service_id'].isin(wednesday_service_ids)]
# Load the stop_times.txt file
stop_times_df = pd.read_csv(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\Lombardia GTFS\AltriGestori\stop_times.txt")
# Filter stop_times to only include rows with trip_id that are in wednesday_trips
wednesday_stop_times_others = stop_times_df[stop_times_df['trip_id'].isin(wednesday_trips['trip_id'])]

In [6]:
# Concatenate the dataframes into one
stop_times = pd.concat([wedndesday_stops_atm, wednesday_stop_times_train, wednesday_stop_times_others])

# If you want to ignore the index and just want to append the dataframes on top of each other
stop_times = pd.concat([wedndesday_stops_atm, wednesday_stop_times_train, wednesday_stop_times_others], ignore_index=True)

# Now you have a combined dataframe named stop_times

## Calculate N-02 for Trains

In [ ]:
# List of GeoDataFrames to merge
gdf_list = [metro_gdf, tram_gdf, bus_gdf, bus_tranne_90_gdf, regional_bus_gdf]
# Merge all the GeoDataFrames into one using pd.concat
merged_gdf = pd.concat(gdf_list, ignore_index=True)
merged_gdf = merged_gdf[['stop_id', 'stop_name', 'geometry']]
# Step 1: Limit merged_gdf to the desired columns
limited_gdf = merged_gdf[['stop_id', 'stop_name', 'geometry']]
# Ensure that train_gdf's CRS matches the merged_gdf's CRS
train_gdf = train_gdf.to_crs(merged_gdf.crs)

In [ ]:
def calculate_nearest(row, other_gdf):
    # Use the unary_union of the other_gdf to create a single geometry that represents all points
    other_points = other_gdf['geometry'].unary_union
    # Find the nearest point in other_points to the geometry of the row
    nearest = other_gdf['geometry'] == nearest_points(row['geometry'], other_points)[1]
    # Get the 'stop_id' of the nearest point
    nearest_stop_id = other_gdf[nearest]['stop_id'].iloc[0]
    # Calculate the distance to the nearest point in meters
    nearest_distance = row['geometry'].distance(other_gdf[nearest]['geometry'].iloc[0])
    # Return a Series with 'nearest_stop_id' and 'nearest_distance'
    return pd.Series([nearest_stop_id, nearest_distance], index=['nearest_stop_id', 'nearest_distance'])

# Ensure the CRS is set to a metric system for distance calculation (if necessary)
# limited_gdf = limited_gdf.to_crs(epsg=32632)

# Apply the function to each row in the train stations GeoDataFrame
# This will add 'nearest_stop_id' and 'nearest_distance' columns to the train_gdf
train_gdf[['nearest_stop_id', 'nearest_distance']] = train_gdf.apply(calculate_nearest, other_gdf=limited_gdf, axis=1)

In [ ]:
# Speed in km/h converted to m/s (4 km/h * 1000 m/km / 3600 s/h)
speed_m_s = 4.5 * 1000 / 3600

# Start time at 8:30 AM
start_time = pd.Timestamp('08:30:00')

def calculate_arrival_time(distance, speed):
    # Time in seconds to travel the distance at the given speed
    travel_time_seconds = distance / speed
    # Calculate arrival time by adding the travel time to the start time
    arrival_time = start_time + pd.Timedelta(seconds=travel_time_seconds)
    # Format the arrival time as HH:MM:SS
    return arrival_time.strftime('%H:%M:%S')

# Apply the function to each row in the DataFrame
train_gdf['arrival_time'] = train_gdf['nearest_distance'].apply(calculate_arrival_time, speed=speed_m_s)

# Now train_gdf['arrival_time'] column will have the time of arrival in GTFS format


In [ ]:
train_gdf['nearest_stop_id'] = train_gdf['nearest_stop_id'].astype(stop_times['stop_id'].dtype)

# Ensure that departure_time in stop_times and arrival_time in train_gdf are both of type datetime.time
stop_times['departure_time'] = pd.to_datetime(stop_times['departure_time']).dt.time
train_gdf['arrival_time'] = pd.to_datetime(train_gdf['arrival_time']).dt.time

# Process for each row in train_gdf to find the next higher departure time
for index, row in train_gdf.iterrows():
    nearest_stop_id = row['nearest_stop_id']
    arrival_time = row['arrival_time']
    
    # Filter stop_times where nearest_stop_id matches stop_id
    filtered_stop_times = stop_times[stop_times['stop_id'] == nearest_stop_id]
    
    # Sort the filtered times by departure_time
    filtered_stop_times_sorted = filtered_stop_times.sort_values('departure_time')
    
    # Find the next higher departure_time value
    next_departure_time = filtered_stop_times_sorted[filtered_stop_times_sorted['departure_time'] > arrival_time]['departure_time'].min()
    
    # Assign the next higher departure_time value back to the train_gdf
    train_gdf.loc[index, 'next_departure_time'] = next_departure_time
# The train_gdf now has a new column 'next_departure_time' with the next higher departure times

In [ ]:
# Define walking speed in meters per minute (4.5 km/h is 75 m/min)
walking_speed_m_per_min = 4.5 * 1000 / 60

# Calculate walking time in minutes and create a new column
train_gdf['walking_time'] = train_gdf['nearest_distance'] / walking_speed_m_per_min

# Convert arrival_time to datetime.time
train_gdf['arrival_time'] = pd.to_datetime(train_gdf['arrival_time'], format='%H:%M:%S').dt.time

# Convert next_departure_time to datetime.time if it's not already
train_gdf['next_departure_time'] = pd.to_datetime(train_gdf['next_departure_time'], format='%H:%M:%S').dt.time

# Define a function to calculate waiting time in minutes
def calculate_waiting_time(arrival_time, next_departure_time):
    # Check for NaN values and return NaN if either time is not a valid string
    if pd.isnull(arrival_time) or pd.isnull(next_departure_time):
        return pd.NaT
    
    # Convert times to datetime.timedelta
    arrival_td = pd.to_timedelta(f'{arrival_time.hour}:{arrival_time.minute}:{arrival_time.second}')
    departure_td = pd.to_timedelta(f'{next_departure_time.hour}:{next_departure_time.minute}:{next_departure_time.second}')
    
    # Calculate the difference in minutes
    waiting_td = departure_td - arrival_td
    return waiting_td.total_seconds() / 60


# Calculate waiting time and create a new column
train_gdf['waiting_time'] = train_gdf.apply(lambda row: calculate_waiting_time(row['arrival_time'], row['next_departure_time']), axis=1)

In [ ]:
train_gdf['N2_2']=train_gdf['walking_time']+train_gdf['waiting_time']
train_gdf = train_gdf.rename(columns={'nearest_distance': 'N2_1'})
# Calculate the mean of the non-NaN values
mean_value = train_gdf['N2_2'].mean()

# Replace NaN values with the mean
train_gdf['N2_2'].fillna(mean_value, inplace=True)

In [ ]:
train=train_gdf[['field_1','stop_id','stop_name','stop_desc','pums','s_lat','s_lon','int_scr','scr_abc','Cluster','catch_area','geometry','N2_1','N2_2']]
train.to_csv(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\PhD\Mapping\ABC\ABC-model-18-02-2024\results\n_02\outputs\n02-train.csv")

## Calculate N-02 for Metro

In [ ]:
# List of GeoDataFrames to merge
gdf_list = [train_gdf, tram_gdf, bus_gdf, bus_tranne_90_gdf, regional_bus_gdf]
# Merge all the GeoDataFrames into one using pd.concat
merged_gdf = pd.concat(gdf_list, ignore_index=True)
merged_gdf = merged_gdf[['stop_id', 'stop_name', 'geometry']]
# Step 1: Limit merged_gdf to the desired columns
limited_gdf = merged_gdf[['stop_id', 'stop_name', 'geometry']]
# Ensure that train_gdf's CRS matches the merged_gdf's CRS
metro_gdf = metro_gdf.to_crs(merged_gdf.crs)

In [ ]:
def calculate_nearest(row, other_gdf):
    # Use the unary_union of the other_gdf to create a single geometry that represents all points
    other_points = other_gdf['geometry'].unary_union
    # Find the nearest point in other_points to the geometry of the row
    nearest = other_gdf['geometry'] == nearest_points(row['geometry'], other_points)[1]
    # Get the 'stop_id' of the nearest point
    nearest_stop_id = other_gdf[nearest]['stop_id'].iloc[0]
    # Calculate the distance to the nearest point in meters
    nearest_distance = row['geometry'].distance(other_gdf[nearest]['geometry'].iloc[0])
    # Return a Series with 'nearest_stop_id' and 'nearest_distance'
    return pd.Series([nearest_stop_id, nearest_distance], index=['nearest_stop_id', 'nearest_distance'])

# Ensure the CRS is set to a metric system for distance calculation (if necessary)
# limited_gdf = limited_gdf.to_crs(epsg=32632)

# Apply the function to each row in the train stations GeoDataFrame
# This will add 'nearest_stop_id' and 'nearest_distance' columns to the train_gdf
metro_gdf[['nearest_stop_id', 'nearest_distance']] = metro_gdf.apply(calculate_nearest, other_gdf=limited_gdf, axis=1)

In [ ]:
# Speed in km/h converted to m/s (4 km/h * 1000 m/km / 3600 s/h)
speed_m_s = 4.5 * 1000 / 3600

# Start time at 8:30 AM
start_time = pd.Timestamp('08:30:00')

def calculate_arrival_time(distance, speed):
    # Time in seconds to travel the distance at the given speed
    travel_time_seconds = distance / speed
    # Calculate arrival time by adding the travel time to the start time
    arrival_time = start_time + pd.Timedelta(seconds=travel_time_seconds)
    # Format the arrival time as HH:MM:SS
    return arrival_time.strftime('%H:%M:%S')

# Apply the function to each row in the DataFrame
metro_gdf['arrival_time'] = metro_gdf['nearest_distance'].apply(calculate_arrival_time, speed=speed_m_s)

# Now train_gdf['arrival_time'] column will have the time of arrival in GTFS format

In [ ]:
metro_gdf['nearest_stop_id'] = metro_gdf['nearest_stop_id'].fillna(-1).astype(stop_times['stop_id'].dtype)

# Ensure that departure_time in stop_times and arrival_time in train_gdf are both of type datetime.time
stop_times['departure_time'] = pd.to_datetime(stop_times['departure_time'], format='%H:%M:%S').dt.time
metro_gdf['arrival_time'] = pd.to_datetime(metro_gdf['arrival_time'], format='%H:%M:%S').dt.time


# Process for each row in train_gdf to find the next higher departure time
for index, row in metro_gdf.iterrows():
    nearest_stop_id = row['nearest_stop_id']
    arrival_time = row['arrival_time']
    
    # Filter stop_times where nearest_stop_id matches stop_id
    filtered_stop_times = stop_times[stop_times['stop_id'] == nearest_stop_id]
    
    # Sort the filtered times by departure_time
    filtered_stop_times_sorted = filtered_stop_times.sort_values('departure_time')
    
    # Find the next higher departure_time value
    next_departure_time = filtered_stop_times_sorted[filtered_stop_times_sorted['departure_time'] > arrival_time]['departure_time'].min()
    
    # Assign the next higher departure_time value back to the train_gdf
    metro_gdf.loc[index, 'next_departure_time'] = next_departure_time
# The train_gdf now has a new column 'next_departure_time' with the next higher departure times

In [ ]:
# Define walking speed in meters per minute (4.5 km/h is 75 m/min)
walking_speed_m_per_min = 4.5 * 1000 / 60

# Calculate walking time in minutes and create a new column
metro_gdf['walking_time'] = metro_gdf['nearest_distance'] / walking_speed_m_per_min

# Convert arrival_time to datetime.time
metro_gdf['arrival_time'] = pd.to_datetime(metro_gdf['arrival_time'], format='%H:%M:%S').dt.time

# Convert next_departure_time to datetime.time if it's not already
metro_gdf['next_departure_time'] = pd.to_datetime(metro_gdf['next_departure_time'], format='%H:%M:%S').dt.time

# Define a function to calculate waiting time in minutes
def calculate_waiting_time(arrival_time, next_departure_time):
    # Check for NaN values and return NaN if either time is not a valid string
    if pd.isnull(arrival_time) or pd.isnull(next_departure_time):
        return pd.NaT
    
    # Convert times to datetime.timedelta
    arrival_td = pd.to_timedelta(f'{arrival_time.hour}:{arrival_time.minute}:{arrival_time.second}')
    departure_td = pd.to_timedelta(f'{next_departure_time.hour}:{next_departure_time.minute}:{next_departure_time.second}')
    
    # Calculate the difference in minutes
    waiting_td = departure_td - arrival_td
    return waiting_td.total_seconds() / 60


# Calculate waiting time and create a new column
metro_gdf['waiting_time'] = metro_gdf.apply(lambda row: calculate_waiting_time(row['arrival_time'], row['next_departure_time']), axis=1)


In [ ]:
metro_gdf['N2_2']=metro_gdf['walking_time']+metro_gdf['waiting_time']
metro_gdf = metro_gdf.rename(columns={'nearest_distance': 'N2_1'})
# Calculate the mean of the non-NaN values
mean_value = metro_gdf['N2_2'].mean()

# Replace NaN values with the mean
metro_gdf['N2_2'].fillna(mean_value, inplace=True)

In [ ]:
metro=metro_gdf[['field_1','stop_id','stop_name','stop_desc','pums','s_lat','s_lon','int_scr','scr_abc','Cluster','catch_area','geometry','N2_1','N2_2']]
metro.to_csv(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\PhD\Mapping\ABC\ABC-model-18-02-2024\results\n_02\outputs\n02-metro.csv")

## Calculate N-02 for Tram

In [22]:
# List of GeoDataFrames to merge
gdf_list = [train_gdf, metro_gdf, bus_gdf, bus_tranne_90_gdf, regional_bus_gdf]
# Merge all the GeoDataFrames into one using pd.concat
merged_gdf = pd.concat(gdf_list, ignore_index=True)
merged_gdf = merged_gdf[['stop_id', 'stop_name', 'geometry']]
# Step 1: Limit merged_gdf to the desired columns
limited_gdf = merged_gdf[['stop_id', 'stop_name', 'geometry']]
# Ensure that train_gdf's CRS matches the merged_gdf's CRS
tram_gdf = tram_gdf.to_crs(merged_gdf.crs)

In [ ]:
def calculate_nearest(row, other_gdf):
    # Use the unary_union of the other_gdf to create a single geometry that represents all points
    other_points = other_gdf['geometry'].unary_union
    # Find the nearest point in other_points to the geometry of the row
    nearest = other_gdf['geometry'] == nearest_points(row['geometry'], other_points)[1]
    # Get the 'stop_id' of the nearest point
    nearest_stop_id = other_gdf[nearest]['stop_id'].iloc[0]
    # Calculate the distance to the nearest point in meters
    nearest_distance = row['geometry'].distance(other_gdf[nearest]['geometry'].iloc[0])
    # Return a Series with 'nearest_stop_id' and 'nearest_distance'
    return pd.Series([nearest_stop_id, nearest_distance], index=['nearest_stop_id', 'nearest_distance'])

# Ensure the CRS is set to a metric system for distance calculation (if necessary)
# limited_gdf = limited_gdf.to_crs(epsg=32632)

# Apply the function to each row in the train stations GeoDataFrame
# This will add 'nearest_stop_id' and 'nearest_distance' columns to the train_gdf
tram_gdf[['nearest_stop_id', 'nearest_distance']] = tram_gdf.apply(calculate_nearest, other_gdf=limited_gdf, axis=1)

In [ ]:
# Speed in km/h converted to m/s (4 km/h * 1000 m/km / 3600 s/h)
speed_m_s = 4.5 * 1000 / 3600

# Start time at 8:30 AM
start_time = pd.Timestamp('08:30:00')

def calculate_arrival_time(distance, speed):
    # Time in seconds to travel the distance at the given speed
    travel_time_seconds = distance / speed
    # Calculate arrival time by adding the travel time to the start time
    arrival_time = start_time + pd.Timedelta(seconds=travel_time_seconds)
    # Format the arrival time as HH:MM:SS
    return arrival_time.strftime('%H:%M:%S')

# Apply the function to each row in the DataFrame
tram_gdf['arrival_time'] = tram_gdf['nearest_distance'].apply(calculate_arrival_time, speed=speed_m_s)

# Now train_gdf['arrival_time'] column will have the time of arrival in GTFS format

In [ ]:
tram_gdf['nearest_stop_id'] = tram_gdf['nearest_stop_id'].fillna(-1).astype(stop_times['stop_id'].dtype)

# Ensure that departure_time in stop_times and arrival_time in train_gdf are both of type datetime.time
stop_times['departure_time'] = pd.to_datetime(stop_times['departure_time'], format='%H:%M:%S').dt.time
tram_gdf['arrival_time'] = pd.to_datetime(tram_gdf['arrival_time'], format='%H:%M:%S').dt.time


# Process for each row in train_gdf to find the next higher departure time
for index, row in tram_gdf.iterrows():
    nearest_stop_id = row['nearest_stop_id']
    arrival_time = row['arrival_time']
    
    # Filter stop_times where nearest_stop_id matches stop_id
    filtered_stop_times = stop_times[stop_times['stop_id'] == nearest_stop_id]
    
    # Sort the filtered times by departure_time
    filtered_stop_times_sorted = filtered_stop_times.sort_values('departure_time')
    
    # Find the next higher departure_time value
    next_departure_time = filtered_stop_times_sorted[filtered_stop_times_sorted['departure_time'] > arrival_time]['departure_time'].min()
    
    # Assign the next higher departure_time value back to the train_gdf
    tram_gdf.loc[index, 'next_departure_time'] = next_departure_time
# The train_gdf now has a new column 'next_departure_time' with the next higher departure times

In [ ]:
# Define walking speed in meters per minute (4.5 km/h is 75 m/min)
walking_speed_m_per_min = 4.5 * 1000 / 60

# Calculate walking time in minutes and create a new column
tram_gdf['walking_time'] = tram_gdf['nearest_distance'] / walking_speed_m_per_min

# Convert arrival_time to datetime.time
tram_gdf['arrival_time'] = pd.to_datetime(tram_gdf['arrival_time'], format='%H:%M:%S').dt.time

# Convert next_departure_time to datetime.time if it's not already
tram_gdf['next_departure_time'] = pd.to_datetime(tram_gdf['next_departure_time'], format='%H:%M:%S').dt.time

# Define a function to calculate waiting time in minutes
def calculate_waiting_time(arrival_time, next_departure_time):
    # Check for NaN values and return NaN if either time is not a valid string
    if pd.isnull(arrival_time) or pd.isnull(next_departure_time):
        return pd.NaT
    
    # Convert times to datetime.timedelta
    arrival_td = pd.to_timedelta(f'{arrival_time.hour}:{arrival_time.minute}:{arrival_time.second}')
    departure_td = pd.to_timedelta(f'{next_departure_time.hour}:{next_departure_time.minute}:{next_departure_time.second}')
    
    # Calculate the difference in minutes
    waiting_td = departure_td - arrival_td
    return waiting_td.total_seconds() / 60


# Calculate waiting time and create a new column
tram_gdf['waiting_time'] = tram_gdf.apply(lambda row: calculate_waiting_time(row['arrival_time'], row['next_departure_time']), axis=1)


In [ ]:
tram_gdf['N2_2']=tram_gdf['walking_time']+tram_gdf['waiting_time']
# Calculate the mean of the non-NaN values
mean_value = tram_gdf['N2_2'].mean()

# Replace NaN values with the mean
tram_gdf['N2_2'].fillna(mean_value, inplace=True)

tram_gdf = tram_gdf.rename(columns={'nearest_distance': 'N2_1'})
tram_gdf

In [ ]:
tram_gdf

In [ ]:
tram=tram_gdf[['field_1','stop_id','stop_name','stop_desc','pums','s_lat','s_lon','int_scr','scr_abc','Cluster','catch_area','geometry','N2_1','N2_2']]
tram.to_csv(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\PhD\Mapping\ABC\ABC-model-18-02-2024\results\n_02\outputs\n02-tram.csv")

## Calculate N-02 for Bus

In [7]:
# List of GeoDataFrames to merge
gdf_list = [train_gdf, metro_gdf, tram_gdf, bus_tranne_90_gdf, regional_bus_gdf]
# Merge all the GeoDataFrames into one using pd.concat
merged_gdf = pd.concat(gdf_list, ignore_index=True)
merged_gdf = merged_gdf[['stop_id', 'stop_name', 'geometry']]
# Step 1: Limit merged_gdf to the desired columns
limited_gdf = merged_gdf[['stop_id', 'stop_name', 'geometry']]
# Ensure that train_gdf's CRS matches the merged_gdf's CRS
bus_gdf = bus_gdf.to_crs(merged_gdf.crs)

In [8]:
def calculate_nearest(row, other_gdf):
    # Use the unary_union of the other_gdf to create a single geometry that represents all points
    other_points = other_gdf['geometry'].unary_union
    # Find the nearest point in other_points to the geometry of the row
    nearest = other_gdf['geometry'] == nearest_points(row['geometry'], other_points)[1]
    # Get the 'stop_id' of the nearest point
    nearest_stop_id = other_gdf[nearest]['stop_id'].iloc[0]
    # Calculate the distance to the nearest point in meters
    nearest_distance = row['geometry'].distance(other_gdf[nearest]['geometry'].iloc[0])
    # Return a Series with 'nearest_stop_id' and 'nearest_distance'
    return pd.Series([nearest_stop_id, nearest_distance], index=['nearest_stop_id', 'nearest_distance'])

# Ensure the CRS is set to a metric system for distance calculation (if necessary)
# limited_gdf = limited_gdf.to_crs(epsg=32632)

# Apply the function to each row in the train stations GeoDataFrame
# This will add 'nearest_stop_id' and 'nearest_distance' columns to the train_gdf
bus_gdf[['nearest_stop_id', 'nearest_distance']] = bus_gdf.apply(calculate_nearest, other_gdf=limited_gdf, axis=1)

In [9]:
# Speed in km/h converted to m/s (4 km/h * 1000 m/km / 3600 s/h)
speed_m_s = 4.5 * 1000 / 3600

# Start time at 8:30 AM
start_time = pd.Timestamp('08:30:00')

def calculate_arrival_time(distance, speed):
    # Time in seconds to travel the distance at the given speed
    travel_time_seconds = distance / speed
    # Calculate arrival time by adding the travel time to the start time
    arrival_time = start_time + pd.Timedelta(seconds=travel_time_seconds)
    # Format the arrival time as HH:MM:SS
    return arrival_time.strftime('%H:%M:%S')

# Apply the function to each row in the DataFrame
bus_gdf['arrival_time'] = bus_gdf['nearest_distance'].apply(calculate_arrival_time, speed=speed_m_s)

# Now train_gdf['arrival_time'] column will have the time of arrival in GTFS format

In [15]:
bus_gdf['nearest_stop_id'] = bus_gdf['nearest_stop_id'].fillna(-1).astype(stop_times['stop_id'].dtype)

# Ensure that departure_time in stop_times and arrival_time in train_gdf are both of type datetime.time
stop_times['departure_time'] = pd.to_datetime(stop_times['departure_time'], format='%H:%M:%S').dt.time
bus_gdf['arrival_time'] = pd.to_datetime(bus_gdf['arrival_time'], format='%H:%M:%S').dt.time


# Process for each row in train_gdf to find the next higher departure time
for index, row in bus_gdf.iterrows():
    nearest_stop_id = row['nearest_stop_id']
    arrival_time = row['arrival_time']
    
    # Filter stop_times where nearest_stop_id matches stop_id
    filtered_stop_times = stop_times[stop_times['stop_id'] == nearest_stop_id]
    
    # Sort the filtered times by departure_time
    filtered_stop_times_sorted = filtered_stop_times.sort_values('departure_time')
    
    # Find the next higher departure_time value
    next_departure_time = filtered_stop_times_sorted[filtered_stop_times_sorted['departure_time'] > arrival_time]['departure_time'].min()
    
    # Assign the next higher departure_time value back to the train_gdf
    bus_gdf.loc[index, 'next_departure_time'] = next_departure_time
# The train_gdf now has a new column 'next_departure_time' with the next higher departure times

In [16]:
# Define walking speed in meters per minute (4.5 km/h is 75 m/min)
walking_speed_m_per_min = 4.5 * 1000 / 60

# Calculate walking time in minutes and create a new column
bus_gdf['walking_time'] = bus_gdf['nearest_distance'] / walking_speed_m_per_min

# Convert arrival_time to datetime.time
bus_gdf['arrival_time'] = pd.to_datetime(bus_gdf['arrival_time'], format='%H:%M:%S').dt.time

# Convert next_departure_time to datetime.time if it's not already
bus_gdf['next_departure_time'] = pd.to_datetime(bus_gdf['next_departure_time'], format='%H:%M:%S').dt.time

# Define a function to calculate waiting time in minutes
def calculate_waiting_time(arrival_time, next_departure_time):
    # Check for NaN values and return NaN if either time is not a valid string
    if pd.isnull(arrival_time) or pd.isnull(next_departure_time):
        return pd.NaT
    
    # Convert times to datetime.timedelta
    arrival_td = pd.to_timedelta(f'{arrival_time.hour}:{arrival_time.minute}:{arrival_time.second}')
    departure_td = pd.to_timedelta(f'{next_departure_time.hour}:{next_departure_time.minute}:{next_departure_time.second}')
    
    # Calculate the difference in minutes
    waiting_td = departure_td - arrival_td
    return waiting_td.total_seconds() / 60


# Calculate waiting time and create a new column
bus_gdf['waiting_time'] = bus_gdf.apply(lambda row: calculate_waiting_time(row['arrival_time'], row['next_departure_time']), axis=1)


In [17]:
bus_gdf['N2_2']=bus_gdf['walking_time']+bus_gdf['waiting_time']
# Calculate the mean of the non-NaN values
mean_value = bus_gdf['N2_2'].mean()

# Replace NaN values with the mean
bus_gdf['N2_2'].fillna(mean_value, inplace=True)

bus_gdf = bus_gdf.rename(columns={'nearest_distance': 'N2_1'})

In [19]:
bus=bus_gdf[['field_1','stop_id','stop_name','stop_desc','s_lat','s_lon','int_scr','scr_abc','Cluster','catch_area','geometry','N2_1','N2_2']]
bus.to_csv(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\PhD\Mapping\ABC\ABC-model-18-02-2024\results\n_02\outputs\n02-bus.csv")